In [0]:
!kill -9 -1

In [1]:
!pip install kaggle

  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /content/.cache/pip/wheels/a4/29/3e/1065ec22dad7255300b56c56611abeb064300be05ae4bce5ed
Successfully built kaggle


In [0]:
from googleapiclient.discovery import build

In [4]:
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname( filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0] ['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [5]:
!kaggle competitions download -c bronchitis -p /content/kaggle

Train_Labels.csv: Downloaded 6KB of 6KB
Sample_Submission.csv: Downloaded 2KB of 2KB
Test_Images.zip: Downloaded 823MB of 823MB
Train_Images.zip: Downloaded 3GB of 3GB


In [6]:
cd kaggle

/content/kaggle


In [7]:
ls

Sample_Submission.csv  Test_Images.zip  Train_Images.zip  Train_Labels.csv


In [0]:
import zipfile

zip_ref = zipfile.ZipFile('Test_Images.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [0]:
import zipfile

zip_ref = zipfile.ZipFile('Train_Images.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [3]:
cd kaggle

/content/kaggle


In [4]:
import pandas as pd

df=pd.read_csv('Train_Labels.csv')
df

,Images,Labels
0,CR_0258,0
1,CR_0467,1
2,CR_0627,1
3,CR_0516,1
4,CR_0496,1
5,CR_0415,1
6,CR_0257,0
7,CR_0255,0
8,CR_0125,0
9,CR_0176,0


In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
import torch.nn as nn


class MnistCNNModel(nn.Module):
    def __init__(self):
        super(MnistCNNModel, self).__init__()
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=4)
        self.fc1 = nn.Linear(61504, 10)

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.maxpool1(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out

In [9]:
from PIL import Image

def register_extension(id, extension):
    Image.EXTENSION[extension.lower()] = id.upper()
Image.register_extension = register_extension
def register_extensions(id, extensions):
    for extension in extensions:
        register_extension(id, extension)
Image.register_extensions = register_extensions

NameError: ignored

In [0]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data.dataset import Dataset  # For custom datasets

class CustomDatasetFromImages(Dataset):
    def __init__(self, csv_path,img_path):
        """
        Args:
            csv_path (string): path to csv file
            img_path (string): path to the folder where images are
            transform: pytorch transforms for transforms and tensor conversion
        """
        # Transforms
        self.to_tensor = transforms.Compose([transforms.Resize((255,255)),transforms.ToTensor()])
        # Read the csv file
        self.data_info = pd.read_csv(csv_path)
        # First column contains the image paths
        self.image_arr = np.asarray(img_path + self.data_info.iloc[:, 0]+'.png')
        # Second column is the labels
        self.label_arr = np.asarray(self.data_info.iloc[:, 1])
        # Calculate len
        self.data_len = len(self.data_info.index)

    def __getitem__(self, index):
        # Get image name from the pandas df
        single_image_name = self.image_arr[index]
        # Open image
        img_as_img = Image.open(single_image_name)

        # Check if there is an operation
        img_as_tensor = self.to_tensor(img_as_img)

        # Get label(class) of the image based on the cropped pandas column
        single_image_label = self.label_arr[index]

        return (img_as_tensor, single_image_label)

    def __len__(self):
        return self.data_len




In [0]:
    transformations = transforms.Compose([transforms.ToTensor()])
    
    custom_mnist_from_images =  \
        CustomDatasetFromImages('Train_Labels.csv','Train_Images/')

    mn_dataset_loader = torch.utils.data.DataLoader(dataset=custom_mnist_from_images,
                                                    batch_size=10,
                                                    shuffle=False)

    model = MnistCNNModel()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

    for i, (images, labels) in enumerate(mn_dataset_loader):
        images = Variable(images)
        labels = Variable(labels)
        # Clear gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(images)
        # Calculate loss
        loss = criterion(outputs, labels)
        # Backward pass
        loss.backward()
        # Update weights
        optimizer.step()
        break

In [183]:
epochs = 10
for epoch in range(1, epochs + 1):
    train(epoch)


Train Epoch: 1 [0/640 (0%)]	


RuntimeError: ignored

In [0]:
import string
   
csv = open("Test_Labels1.csv", "w") 
columnTitleRow = "Images, Labels\n"
csv.write(columnTitleRow)
for filename in os.listdir('Test_Images/'):
   csv.write(str.replace(filename,'.png','')+","+"0"+"\n")


In [163]:
df=pd.read_csv("Train_Labels.csv")
df[:1]

,Images,Labels
0,CR_0258,0


In [164]:
 df_t=pd.read_csv("Test_Labels1.csv")
 df_t[:1]

,Images,Labels
0,MR_0372,0


In [175]:
    transformations = transforms.Compose([transforms.ToTensor()])
    
    custom_mnist_from_images_t =  \
        CustomDatasetFromImages('Test_Labels1.csv','Test_Images/')

    mn_dataset_loader_t = torch.utils.data.DataLoader(dataset=custom_mnist_from_images_t,
                                                    batch_size=1,
                                                    shuffle=False)
    
    model.eval()
    for i, (images, labels) in enumerate(mn_dataset_loader_t):
        if 3 not in images.shape:
          images = Variable(images)
          labels = Variable(labels)
          output = model(images)
          pred = output.data.max(1)[1] # get the index of the max log-probability
          print(str(i)+","+str(pred[0]))
        else:
          print("not processed-"+str(i)+","+"0")
        

0,1
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0
10,0
11,0
12,1
13,0
14,0
15,1
16,0
17,0
18,0
19,0
not processed-20,0
21,0
22,1
23,0
24,0
25,0
26,0
27,1
28,0
29,1
30,0
31,0
32,0
33,0
34,0
35,0
36,0
37,0
38,0
39,1
40,1
41,1
42,0
43,1
44,0
45,0
46,0
47,0
48,0
49,0
50,0
51,0
52,0
53,1
54,0
55,0
56,0
57,0
58,1
59,0
60,0
61,0
62,0
63,0
not processed-64,0
65,0
66,0
67,1
68,0
69,0
70,0
71,0
72,1
73,0
74,0
75,1
76,0
77,0
78,1
79,0
80,1
81,1
82,1
83,0
84,0
85,0
86,0
87,0
88,0
89,1
90,0
91,0
92,0
93,0
94,0
95,0
96,0
97,0
98,1
99,0
100,0
101,1
102,0
103,1
104,0
not processed-105,0
106,0
107,1
108,0
109,1
110,0
111,0
112,0
113,1
114,0
115,1
116,1
117,0
118,1
119,0
120,0
121,1
122,1
123,1
124,1
125,0
not processed-126,0
127,0
128,0
129,1
not processed-130,0
131,0
132,1
133,0
134,0
135,0
136,0
137,0
138,0
139,0
140,0
141,0
142,1
143,1
144,0
145,0
146,0
147,1
148,0
149,1
150,0
151,0
152,0
153,0
154,0
155,0
156,0
not processed-157,0
158,1
159,0
